In [38]:
# -----------------------------------------------------------------------------
# Hick-Hyman Law, human performance principle depicting choice-reaction time
# -----------------------------------------------------------------------------
import tkinter as tk
from tkinter import messagebox
import matplotlib.pyplot as plt
import numpy as np
import time as tm

def onHelpClicked():
    tk.messagebox.showinfo("Usage Help", "blah blah...")

def onStartClicked():
    print ("ready, start!")

def onButtonClicked(event):
    print ("Button Clicked!")

root = tk.Tk()
root.title("Hick-Hyman Law")
root.resizable(width=False, height=False)
# GUI Layout
frameExplain = tk.Frame(root)
frameGameArea = tk.Frame(root, bg='white', bd=2, relief=tk.GROOVE, width=500, height=300)
frameBonus = tk.Frame(root)
frameExplain.grid(row=0, column=0, padx=10, pady=10)
frameGameArea.grid(row=1, column=0, padx=10, pady=10)
frameGameArea.grid_propagate(0)
frameBonus.grid(row=0, column=1, padx=10, pady=10)
buttonHelp = tk.Button(frameExplain, text="Usage Help", command=onHelpClicked)
buttonStart = tk.Button(frameExplain, text="Start", command=onStartClicked)
labelPause = tk.Label(frameExplain, text="Press <space> key to pause at any time")
labelRequirement = tk.Label(frameBonus, text="Requirement here")
labelFeedback = tk.Label(frameBonus, text="chronoscope or hitting points here", bd=2, relief=tk.RAISED)
buttonHelp.grid(row=0, column=0, ipadx=10, ipady=10, padx=10, sticky=tk.W)
buttonStart.grid(row=0, column=1, ipadx=10, ipady=10, padx=10, sticky=tk.W)
labelPause.grid(row=0, column=2, padx=10, sticky=tk.W)
labelRequirement.grid(row=0, column=0, padx=10, pady=10)
labelFeedback.grid(row=1, column=0, ipadx=10, ipady=10, padx=10, pady=10)
# define alternatives
alternativesKey = ['A', 'E', 'R', 'G', 'H', 'U', 'I', 'L']
alternativesButton = [0 for x in range(7)]
for x in range(7):
    alternativesButton[x] = tk.Button(frameGameArea, text=alternativesKey[x], width=7, height=2)
    alternativesButton[x].grid(row=x, column=0)
    alternativesButton[x].bind('<Button-1>', onButtonClicked)
# hit-key experiment (maximum 8 keys -- aerg huil)
# generate random alternatives with key label and colour


# have test subject hit keys to respond

# experiment 1: require accuracy, vary the amount of alternatives, adjust flying speed
# Reaction Time ~ f(amount of alternatives)



# experiment 2: require speed, 8 alternatives, high flying speed
# Reaction Time ~ f(error rate)




# empirical data plotting

# result interpretation

root.mainloop()
